![](../images/rivacon_frontmark_combined_header.svg)

In [ ]:
import datetime
from dateutil.relativedelta import relativedelta
import pyvacon.analytics as analytics
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.marketdata.plot as mkt_plot
import pyvacon.marketdata.bootstrapping as bootstr
import math
import pandas as pd
import qgrid
import pyvacon
# the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
import matplotlib.pyplot as plt

# alternative library for plots (offline version)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *

init_notebook_mode(connected=True) 

# Cross-Currency Swap

## Definition of a Cross-Currency Swap

A basic cross-currency swap involves the exchange of a notional and interest payments in one currvency against the notional and interest payments in another currency. The notionals are usually exchanged at the beginning and the end of the life of the swap. The rates can either be both floating, both fixed or a mixture of floating and fixed.

## Valuation of Cross-Currency Swaps

As in plain vanilla interest-rate swaps, the valuation of cross-currency swap (CCS) can either be undertaken in terms of the difference between two bonds or as a portfolio of forward contracts. Analogous to the interest-rate swap, RIVACON uses the latter method.

**Example:** Suppose a flat term structure of LIBOR/swap interest rates with 5% in country A and 6% in country B (continously compounding). Some time ago, a financial institution has entered into a currency swap where it receives 7% in currency B and pays 4% in currency A. The principals in currency A are 10 million and in currency B 1,000 million. The maturity of the swap is in three years and the current exchange rate of currency A/B is 1/100. 

### Valuation in Terms of Bond Prices

If we price the CCS in terms of bond prices, we need to calculate the difference in the present values of both legs, where the foreign currency leg (here leg B) has to be converted into the domestic currency (here currency A). 
Hence, the pay-leg has cash-flows of 0.4 million in currency A in each year and the principal of 10 million in the last year, the receive leg cash-flows are 70 million of currency B in each year and the principal of 1,000 million in the final year. Both cash flows are discounted with the currency-specific disount-factors. Finally, the present value of the cash-flows of the foreign-currency leg (leg B) has to be converted into currency A. The CCS value is then the difference of the present values in the same currency.

In [ ]:
pay_dates = [1,2,3]
r_dsc_A = 0.05
r_dsc_B = 0.06

fx_spot = 1/100

#Discount factors 
df_A = []
for i in range(len(pay_dates)):
    df_A.append(math.exp(-r_dsc_A*pay_dates[i]))
    
df_B = []
for i in range(len(pay_dates)):
    df_B.append(math.exp(-r_dsc_B*pay_dates[i]))
    
#Pay leg
CF_pay = [0.4, 0.4, 10.4]
PV_pay = []
for i in range(len(CF_pay)):
    PV_pay.append(CF_pay[i]*df_A[i])

#Receive Leg
CF_receive = [70, 70, 1070]
PV_receive = []
for i in range(len(CF_receive)):
    PV_receive.append(CF_receive[i]*df_B[i])
    
CCS_value = (-sum(PV_pay)+sum(PV_receive)*fx_spot)
print('Value of the Cross-Currency Swap =',CCS_value)

### Valuation in Terms of Forward-Rate Agreements

In order to price the CCS in terms of forward rate agreements, we need to determine the cash flows of the pay- and receive leg in currency a and discount the net-cash-flows. For the pay-leg, the cash flows are 0.4 million in currency A in each year and the principal of 10 million in the last year. The cash-flows of the receive leg are 70 million of currency B in each year and the principal of 1,000 million in the final year. These have to be multiplied with the forward-exchange-rates for the relevant maturity. Finally, the net cash-flow for each period has to be discounted and summed up to the value of the CCS. The pricing of pricing of the CCS in terms of forward-rate agreements is described in the following code section.

In [ ]:
pay_dates = [1,2,3]
r_dsc_A = 0.05
r_dsc_B = 0.06

# Exchange rates
fx_spot = 1/100
fwd_fx_rates = []
for i in range(len(pay_dates)):
    fwd_fx_rates.append(fx_spot*math.exp((r_dsc_A-r_dsc_B)*pay_dates[i]))

#Discount factors 
df_A = []
for i in range(len(pay_dates)):
    df_A.append(math.exp(-r_dsc_A*pay_dates[i]))

#Pay leg
CF_pay = [0.4, 0.4, 10.4]

#Receive Leg
CF_receive = [70, 70, 1070]

CF_receive_cur_A = []
for i in range(len(fwd_fx_rates)):
    CF_receive_cur_A.append(CF_receive[i]*fwd_fx_rates[i])

# Valuation as difference between Pay- and Receive leg
CF_CCS_Value = []
for i in range(len(CF_pay)):
    CF_CCS_Value.append(-CF_pay[i]+CF_receive_cur_A[i])

PV_CCS_Value = []
for i in range(len(CF_pay)):
    PV_CCS_Value.append(CF_CCS_Value[i]*df_A[i])
CCS_value = sum(PV_CCS_Value)

print('Value of the Cross-Currency Swap =', CCS_value)

## Cross-Currency Swap - Sample Data Anton

### Calibration of discount curves
We now present the calibration of EUR curves based on actual market data. The input quotes as well as the instrument definition and conventions are provided in an input csv. file and are loaded into a pandas data frame object:

In [ ]:
refdate_d = datetime.datetime(2018,8,31,0,0)
refdate = refdate = converter.getLTime(refdate_d)
holidays = analytics.SimpleHolidayCalendar('GER_HOL')
#holidays.setWeekdayAsHoliday(0) # set sunday as holiday
#holidays.setWeekdayAsHoliday(6) #set saturday as holidays

# set directory and file name for Input Quotes
dirName = "../inputdata/"
fileName = "/inputQuotes.csv"

# get instrument quotes and conventions from input .csv file 
column_names = ['Maturity','Instrument','Currency','Quote','UnderlyingIndex','UnderlyingTenor', 'UnderlyingPaymentFrequency',
                'BasisIndex','BasisTenor','BasisPaymentFrequency','PaymentFrequencyFixed','DayCountFixed',
                'DayCountFloat','DayCountBasis','RollConventionFixed','RollConventionFloat','RollConventionBasis', 'SpotLag']

dfQuotes = pd.read_csv(dirName + fileName, sep= ";", decimal =",", skiprows=[0], header=None, names = column_names)

# show data frame
#dfQuotes

#### EONIA curve

In [ ]:
# get input data for the EONIA curve
dfQuotesOIS = dfQuotes[dfQuotes['UnderlyingIndex'] == 'EONIA']
#print(dfQuotesOIS)

# set up curve parameters for bootstrapping algorithm
eoniaCurveSpec =  {'refDate': refdate_d, 
                   'curveName': 'eonia',
                   'dayCount': enums.DayCounter.ACT365_FIXED,
                   'calendar': holidays}
# get eonia curve             
eoniaCurve = bootstr.bootstrap_curve(dfQuotesOIS,eoniaCurveSpec)

#### 3M EURIBOR curve

In [ ]:
# get input data for the 3M EURIBOR curve
dfQuotes3M = dfQuotes[(dfQuotes['UnderlyingIndex'] == 'EURIBOR') & (dfQuotes['UnderlyingTenor'] == '3M')]

# set up curve parameters for the 3M EURIBOR curve. The eonia curve is used for bootstrapping
euribor3MCurveSpec =  {'refDate': refdate_d, 
                      'curveName': 'euribor_3M',
                      'dayCount': enums.DayCounter.ACT365_FIXED,
                      'calendar': holidays,
                      'discountCurve': eoniaCurve}
# get 3M euribor curve              
euribor3MCurve = bootstr.bootstrap_curve(dfQuotes3M,euribor3MCurveSpec)

#### 6M EURIBOR curve

In [ ]:
# get input data for the 6M EURIBOR curve
dfQuotes6M = dfQuotes[(dfQuotes['UnderlyingIndex'] == 'EURIBOR') & (dfQuotes['UnderlyingTenor'] == '6M')]

# set up curve parameters for the 6M EURIBOR curve
euribor6MCurveSpec =  {'refDate': refdate_d, 
                      'curveName': 'euribor_6M',
                      'dayCount': enums.DayCounter.ACT365_FIXED,
                      'calendar': holidays,
                      'discountCurve': eoniaCurve,
                      'basisCurve': euribor3MCurve}
# get 6M euribor curve             
euribor6MCurve = bootstr.bootstrap_curve(dfQuotes6M,euribor6MCurveSpec)

#### GBP 3M

#### GBP/EUR Collateralized

In [ ]:
#eurodollarcurvecollaterizd fehlt noch

#### Plotting Curves

In [ ]:
# get output discount factors and zero rates for specified maturities
days_to_maturity = [1,12,19,26,35,68,96,127,159,187,217,249,278,370,461,551,643,735,1103,1468,1832,2196,2562,2927,3294,3659,4023,4388,5486,7312,9136,10962,14615,18268]
dates = converter.createPTimeList(refdate, days_to_maturity)
dates_d = converter.create_datetime_list(dates)

df_ois = analytics.vectorDouble()
zr_ois = analytics.vectorDouble()
eoniaCurve.value(df_ois, refdate, dates)

df_3m = analytics.vectorDouble()
zr_3m = analytics.vectorDouble()
euribor3MCurve.value(df_3m, refdate, dates)

df_6m = analytics.vectorDouble()
zr_6m = analytics.vectorDouble()
euribor6MCurve.value(df_6m, refdate, dates)

for i in range(0,len(days_to_maturity)):
    zr_ois.append(-math.log(df_ois[i])/days_to_maturity[i]*365.0)
    zr_3m.append(-math.log(df_3m[i])/days_to_maturity[i]*365.0)
    zr_6m.append(-math.log(df_6m[i])/days_to_maturity[i]*365.0)

# create data frame with curves
rates_list = {'Dates': dates_d, 
              'DiscountFactor_OIS': df_ois,
              'ZeroRate_OIS': zr_ois,
              'DiscountFactor_3M': df_3m,
              'ZeroRate_3M': zr_3m,
              'DiscountFactor_6M': df_6m,
              'ZeroRate_6M': zr_6m}

rates = pd.DataFrame(rates_list, index = days_to_maturity)    

# plot discount factors
df_plot_data = [Scatter(name = 'Eonia', x = rates.index/365, y = rates['DiscountFactor_OIS'], mode = 'lines+markers'),
                Scatter(name = '3M Euribor', x = rates.index/365, y = rates['DiscountFactor_3M'], mode = 'lines+markers' ),
                Scatter(name = '6M Euribor', x = rates.index/365, y = rates['DiscountFactor_6M'], mode = 'lines+markers')]

df_layout = Layout(title = "Discount Factors",
               xaxis=dict(title ='Time to maturity (years)'),
               yaxis=dict(title = 'Discount Factor'))

# plot zero rates
zr_plot_data = [Scatter(name = 'Eonia', x = rates.index/365, y = rates['ZeroRate_OIS'], mode = 'lines+markers'),
                Scatter(name = '3M Euribor', x = rates.index/365, y = rates['ZeroRate_3M'], mode = 'lines+markers'),
                Scatter(name = '6M Euribor', x = rates.index/365, y = rates['ZeroRate_6M'], mode = 'lines+markers')]

zr_layout = Layout(title="Zero Rates",
               xaxis=dict(title = 'Time to maturity (years)'),
               yaxis=dict(title = 'Rate'))

iplot(Figure(data=df_plot_data, layout=df_layout))
iplot(Figure(data=zr_plot_data, layout=zr_layout))

### Valuation of the Cross-Currency Swap

#### Importing CCS specification data from excel

In [ ]:
data_file = "../inputdata/20180920_MX_Econ.xlsm"
sheet = "original"

# Create Pandas Dataframe
original = pd.read_excel(data_file, sheetname=sheet)

#### Create EUR Leg

In [ ]:
# Switch off warnings
pd.options.mode.chained_assignment = None

#EUR_LEG
eur_leg = original[(original['Lg'] == 1) & (original['Start Date'] >= refdate_d)]

fwd_rate = euribor3MCurve
dsc_rate = eoniaCurve

# function for getting discount factor from curve
def getdf(i, col, curve, leg):
    return curve.value(refdate, converter.getLTime(datetime.datetime.strptime(str(leg.iloc[i][col]),"%Y-%m-%d %H:%M:%S")))

#total_rows = len(eur_leg['Start Date'])
#print(total_rows)
#df_start = []
#for i in range(0,total_rows):
#    df_start.append(getdf(i))
#eur_leg['df_start'] = pd.Series(df_start, index=eur_leg.index)

eur_leg['timediff'] = pd.Series(eur_leg['Nb of Days']/360, index=eur_leg.index)
start= eur_leg.index.values[0]
eur_leg['df_start'] = pd.Series(eur_leg.index.values,index=eur_leg.index).apply(lambda row: getdf(row-start, 'Start Date', fwd_rate, eur_leg))
eur_leg['df_end'] = pd.Series(eur_leg.index.values,index=eur_leg.index).apply(lambda row: getdf(row-start, 'End Date', fwd_rate, eur_leg))
eur_leg['rate'] = pd.Series(100*(eur_leg['df_start']/eur_leg['df_end']-1)/eur_leg['timediff']+eur_leg['Margin'])
eur_leg['flow'] = pd.Series(eur_leg['Remaining Capital']*(eur_leg['rate']/100)*eur_leg['timediff'])
eur_leg['disc_fac'] = pd.Series(eur_leg.index.values,index=eur_leg.index).apply(lambda row: getdf(row-start, 'Payment Date', dsc_rate, eur_leg))
eur_leg['PV'] = pd.Series(eur_leg['flow']*eur_leg['disc_fac'])

reset_eur_leg = original[(original['Lg'] ==1) & (original['Flow Tp'] =='PRI') & (original['Sub. Tp']=='FIN')]
reset_eur_leg['disc_fac'] = pd.Series(reset_eur_leg.index.values,index=reset_eur_leg.index).apply(lambda row: getdf(0, 'Payment Date', dsc_rate, reset_eur_leg))
reset_eur_leg['PV'] = pd.Series(-reset_eur_leg['Flow']*reset_eur_leg['disc_fac'])

PV_eur_leg = sum(eur_leg['PV'])+sum(reset_eur_leg['PV'])
print(PV_eur_leg)

#### Create GBP Leg

**hier müssen noch die discount curves angepasst werden**

In [ ]:
#GBP_LEG
gbp_leg = original[(original['Lg'] == 2) & (original['Start Date'] >= refdate_d)]

fwd_rate = euribor3MCurve
dsc_rate = eoniaCurve
fx_spot = 1.12

# function for getting discount factor from curv
def getdf(i, col, curve, leg):
    return curve.value(refdate, converter.getLTime(datetime.datetime.strptime(str(leg.iloc[i][col]),"%Y-%m-%d %H:%M:%S")))

gbp_leg['timediff'] = pd.Series(gbp_leg['Nb of Days']/360, index=gbp_leg.index)
start= gbp_leg.index.values[0]
gbp_leg['df_start'] = pd.Series(gbp_leg.index.values,index=gbp_leg.index).apply(lambda row: getdf(row-start, 'Start Date', fwd_rate, gbp_leg))
gbp_leg['df_end'] = pd.Series(gbp_leg.index.values,index=gbp_leg.index).apply(lambda row: getdf(row-start, 'End Date', fwd_rate, gbp_leg))
gbp_leg['rate'] = pd.Series(100*(gbp_leg['df_start']/gbp_leg['df_end']-1)/gbp_leg['timediff']+gbp_leg['Margin'])
gbp_leg['flow'] = pd.Series(gbp_leg['Remaining Capital']*(gbp_leg['rate']/100)*gbp_leg['timediff'])
gbp_leg['disc_fac'] = pd.Series(gbp_leg.index.values,index=gbp_leg.index).apply(lambda row: getdf(row-start, 'Payment Date', dsc_rate, gbp_leg))
gbp_leg['PV'] = pd.Series(gbp_leg['flow']*gbp_leg['disc_fac'])

reset_gbp_leg = original[(original['Lg'] ==2) & (original['Flow Tp'] =='PRI') & (original['Sub. Tp']=='FIN')]
reset_gbp_leg['disc_fac'] = pd.Series(reset_gbp_leg.index.values,index=reset_gbp_leg.index).apply(lambda row: getdf(0, 'Payment Date', dsc_rate, reset_gbp_leg))
reset_gbp_leg['PV'] = pd.Series(reset_gbp_leg['Flow']*reset_gbp_leg['disc_fac'])

PV_gbp_leg = (sum(gbp_leg['PV'])+sum(reset_gbp_leg['PV']))*fx_spot
print(PV_gbp_leg)

#### Calculating Cross-Currency Swap Value

In [ ]:
#CCS Value
print('Cross-Currency Swap Value = ', PV_gbp_leg-PV_eur_leg)